In [1]:
from fundar.structures import List as Lista
from fundar.utils import snd, load_from_str_or_buf, has, access, identity, is_empty
from fundar import json
from utils import find_file, slicer
from data_transformers import transformer
from data_transformers import chain
from pandas import DataFrame
from glob import glob
import os

def str_split(sep, **kwargs):
    return lambda x: str.split(x, sep,**kwargs)

def bytes_decode(encoding, **kwargs):
    return lambda x: bytes.decode(x, encoding, **kwargs)

In [2]:
mappings = find_file('mappings.json')
transformers = glob('./datasets_update/definitivos/transformers/*.py')
transformers = Lista(transformers)

In [3]:
get_name = (
    os.path.split,  # list[str]
    snd,            # str
    str_split('_'), # list[str]
    slicer(2),      # list[str]
    '_'.join        # str 
) # str

get_name = Lista.of(Lista(get_name).chain, lambda x: x) # (str, str)

name_mappings = {k:v for k,v in transformers.map(get_name.apply)}

In [4]:
files = {y : {'file':k, 'transformer': name_mappings.get(y, None)} 
 for k,v in json.load(mappings).items() 
 for x in v 
 for y in (x['public'], ) # alias
}

In [5]:
source_code = load_from_str_or_buf(files['CAMCLI_g12']['transformer']).readlines()
source_code = Lista(source_code).map(bytes_decode('latin-1'))

def parse_source(source_code: str):
    definitions_range = source_code.which(has('DEFINITIONS'))
    definitions_range[1] += 1

    pipeline_range = source_code.which(has('PIPELINE'))
    pipeline_range[1] += 1

    definitions = source_code[slice(*definitions_range)]
    pipeline = source_code[slice(*pipeline_range)]

    definitions = definitions[1:-1].map(str.rstrip)

    convert_indices = definitions.which(has('@transformer.convert'))
    return_indices = definitions.which(has('return'))

    definitions_indices = convert_indices.zip(return_indices)

    f_definitions = [None for _ in definitions_indices]

    for k,(i,j) in enumerate(definitions_indices):
        f_definitions[k] = '\n'.join(definitions[i+1:j+1])

    pipeline_str = ''.join(pipeline[1:-1]).split('pipeline =')[1].lstrip()

    return pipeline, f_definitions

In [8]:
def transformer_from_source(source: str):
    if not source.startswith('def '):
        raise ValueError('Source string must start with function definition')
    
    left_paren = source.index('(')
    k = len('def ')
    name = source[k:left_paren]
    
    exec(source, globals())
    obj = globals().get(name)
    source = '@transformers.convert\n' + source
    globals()[name] = transformer.convert(obj, name=name, external_sourcelines=source)
    return obj

In [9]:
transformer_from_source(f_definitions[0])

<function __main__.rename_cols(df: pandas.core.frame.DataFrame, map)>

In [13]:
transformer_from_source(f_definitions[1])

<function __main__.drop_col(df: pandas.core.frame.DataFrame, col, axis=1)>

In [31]:
pipeline_str = ''.join(pipeline[1:-1]).split('pipeline =')[1].lstrip()

In [35]:
df = DataFrame({'sector': ['a']})

In [36]:
pipeline = eval(pipeline_str)

In [38]:
pipeline(df)

([(<transformer 'start'>,
   {},
     sector
   0      a),
  (<transformer 'rename_cols'>,
   {'map': {'subsector': 'indicador', 'valor_en_mtco2e': 'valor'},
    'df':   sector
    0      a},
     sector
   0      a),
  (<transformer 'drop_col'>,
   {'col': 'sector', 'axis': 1, 'df':   sector
    0      a},
   Empty DataFrame
   Columns: []
   Index: [0])],
 Empty DataFrame
 Columns: []
 Index: [0])